In [73]:
%%writefile Distance_Oracle_Serial.cpp

#include <vector>
#include <utility>
#include <set>
#include <climits>
#include <stdio.h>
#include <time.h>
#include <string.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>

#define LLI_MAX 0x7fffffffffffffff

#define K 3
#define N 10
#define ERDOS_RENYI_PROBA 0.5
#define ERDOS_RENYI_MAX_WEIGHT 50
#define PRINT_STEPS true

using namespace std;

struct Edge
{
    int head;
    long long int cost;
};

vector<vector<Edge>> generate_graph()
{
    vector<vector<Edge>> g;
    g.resize(N+1);
 
    for (int a = 1; a <= N; a++)
    {
        for (int b = a+1; b <= N; b++)
        {
            double ranv = ((double) rand()) / RAND_MAX;
            if (ranv <= ERDOS_RENYI_PROBA)
            {
                long long int wgt = 1 + (rand() % ERDOS_RENYI_MAX_WEIGHT);
                g[a].push_back({b, wgt});
                g[b].push_back({a, wgt});
            }
        }
    }
    
    return g;
}

vector<vector<Edge>> add_zero_edge(vector<vector<Edge>> g)
{
    for (int i = 1; i < g.size(); i++)
    {
        g[0].push_back({i, 0});
    }

    return g;
}

vector<long long int> bellman_ford(vector<vector<Edge>> &g, int s)
{
    vector<vector<long long int>> memo(g.size()+2, vector<long long int>(g.size(), LLI_MAX));
    memo[0][s] = 0;

    for (int i = 1; i < memo.size(); i++)
    {
        for (int n = 0; n < g.size(); n++)
        {
            if (memo[i-1][n] < memo[i][n])
            {
                memo[i][n] = memo[i-1][n];
            }
         
            for (auto& e : g[n])
            {
                if (memo[i-1][n] != LLI_MAX)
                {
                    if (memo[i-1][n] + e.cost < memo[i][e.head])
                    {
                         memo[i][e.head] = memo[i-1][n] + e.cost;
                    }
                }
            }
        }
    }

    for (int j = 0; j < g.size(); j++)
    {
        if (memo[g.size()+1][j] != memo[g.size()][j])
        {
            printf("NEGATIVE CYCLE FOUND!\n");
        }
    }

    return memo[g.size()];
}

vector<long long int> djikstra(const vector<vector<Edge>> &g, int s)
{
    vector<long long int> dist(g.size(), LLI_MAX);
    set<pair<int, long long int>> frontier;
    frontier.insert({0,s});

    while (!frontier.empty())
    {
        pair<int, long long int> p = *frontier.begin();
        frontier.erase(frontier.begin());

        int d = p.first;
        int n = p.second;
     
        dist[n] = d;

        for (auto e : g[n])
        {
            if (dist[n]+e.cost < dist[e.head])
            {
                if (dist[e.head] != LLI_MAX)
                {
                    frontier.erase(frontier.find({dist[e.head], e.head}));
                }
                frontier.insert({dist[n]+e.cost, e.head});
                dist[e.head] = dist[n]+e.cost;
            }
        }
    }

    return dist;
}

vector<vector<long long int>> johnson(vector<vector<Edge>> &g)
{
    vector<vector<Edge>> gprime = add_zero_edge(g);
    vector<long long int> ssp = bellman_ford(gprime, 0);
 
    for (int i = 1; i < g.size(); i++)
    {
        for (auto &e : g[i])
        {
            e.cost = e.cost + ssp[i] - ssp[e.head];
        }
    }

    vector<vector<long long int>> allsp(g.size());
    for (int i = 1; i < g.size(); i++)
    {
        allsp[i] = djikstra(g, i);
    }

    for (int u = 1; u < g.size(); u++)
    {
        for (int v = 1; v < g.size(); v++)
        {
            if (allsp[u][v] != LLI_MAX)
            {
                allsp[u][v] += ssp[v] - ssp[u];
            }
        }
    }

    return allsp;
}

void create_i_centers_mat(vector<vector<int>> &i_centers, vector<vector<int>> &i_centers_diff)
{
    i_centers.resize(K+1);
    i_centers_diff.resize(K);

    double proba = pow(N, -1.0/K), ranv;
    bool dropped;
    
    for (int a = 1; a <= N; a++)
    {
        dropped = false;
        i_centers[0].push_back(a);
        for (int b = 1; b < K; b++)
        {
            ranv = ((double) rand()) / RAND_MAX;
            if (ranv <= proba)
            {
                i_centers[b].push_back(a);
            }
            else
            {
                dropped = true;
                i_centers_diff[b-1].push_back(a);
                break;
            }
        }
     
        if (!dropped)
        {
            i_centers_diff[K-1].push_back(a);
        }
    }
 
    return;
}

void create_i_centers_min_dist_vert__bunches(vector<vector<int>> &i_centers,
                                             vector<vector<int>> &i_centers_diff,
                                             vector<vector<int>> &i_centers_min_dist_vert,
                                             bool bunches[N+1][N+1],
                                             vector<vector<long long int>> &dist_mat)
{
    i_centers_min_dist_vert.resize(N+1);
 
    for (int a = 1; a <= N; a++)
    {
        i_centers_min_dist_vert[a].push_back(a);
     
        for (int b = 1; b < K; b++)
        {
            long long int min_distance = LLI_MAX;
            int min_vert = -1;
            for (int c = 0; c < i_centers[b].size(); c++)
            {
                if (dist_mat[a][i_centers[b][c]] < min_distance)
                {
                    min_distance = dist_mat[a][i_centers[b][c]];
                    min_vert = i_centers[b][c];
                }
            }
         
            if (min_vert != -1)
            {
                i_centers_min_dist_vert[a].push_back(min_vert);
            }

            for (int c = 0; c < i_centers_diff[b-1].size(); c++)
            {
                if (dist_mat[a][i_centers_diff[b-1][c]] < min_distance)
                {
                    bunches[a][i_centers_diff[b-1][c]] = 1;
                }
            }
        }

        for (int b = 0; b < i_centers_diff[K-1].size(); b++)
        {
            bunches[a][i_centers_diff[K-1][b]] = 1;
        }
    }
}

long long int get_approximate_distance(int u, int v, bool bunches[N+1][N+1],
                                       vector<vector<int>> i_centers_min_dist_vert,
                                       vector<vector<long long int>> dist_mat)
{
    int w = u, i = 0, tmp;
    int tmp_u = u, tmp_v = v;
 
    while (!bunches[tmp_v][w])
    {
        i++;

        tmp = tmp_u;
        tmp_u = tmp_v;
        tmp_v = tmp;

        if (i >= K)
        {
            printf("ERROR! i = %d >= K = %d for u = %d and v = %d\n", i, K, u, v);
            break;
        }
        w = i_centers_min_dist_vert[tmp_u][i];
    }
 
    return dist_mat[w][tmp_u] + dist_mat[w][tmp_v];
}

int main ()
{
    srand(1234);

    vector<vector<Edge>> g = generate_graph();
    if (PRINT_STEPS)
    {
        printf("Randomly Generated Graph:\n");
        for (int a = 1; a <= N; a++)
        {
            printf("V%d: ", a-1);
            for (int b = 0; b < g[a].size(); b++)
            {
                printf("(V%d, %lld) ", g[a][b].head-1, g[a][b].cost);
            }
            printf("\n");
        }
        printf("\n");
    }

    vector<vector<long long int>> dist_mat = johnson(g);
    if (PRINT_STEPS)
    {
        printf("Distance Matrix:\n");
        for (int a = 1; a <= N; a++)
        {
            printf("\tV%d", a-1);
        }
        printf("\n");
        for (int a = 1; a <= N; a++)
        {
            printf("V%d\t", a-1);
            for (int b = 1; b <= N; b++)
            {
                printf("%lld\t", dist_mat[a][b]);
            }
            printf("\n");
        }
        printf("\n");
    }

    auto start = std::chrono::high_resolution_clock::now();
 
    vector<vector<int>> i_centers, i_centers_diff;
    create_i_centers_mat(i_centers, i_centers_diff);
 
 /*
    i_centers[1].resize(5);
    i_centers[1][0]=1; i_centers[1][1]=2; i_centers[1][2]=4; i_centers[1][3]=5; i_centers[1][4]=7; 
 
    i_centers[2].resize(1);
    i_centers[2][0]=2;
 
    i_centers_diff[0].resize(5);
    i_centers_diff[0][0]=3; i_centers_diff[0][1]=6; i_centers_diff[0][2]=8; i_centers_diff[0][3]=9; i_centers_diff[0][4]=10; 
 
    i_centers_diff[1].resize(4);
    i_centers_diff[1][0]=1; i_centers_diff[1][1]=4; i_centers_diff[1][2]=5; i_centers_diff[1][3]=7; 
 
    i_centers_diff[2].resize(1);
    i_centers_diff[2][0]=2; 
 */
 
    if (PRINT_STEPS)
    {
        for (int a = 0; a <= K; a++)
        {
            printf("I Center %d\t: ", a);
            for (int b = 0; b < i_centers[a].size(); b++)
            {
                printf("%d ", i_centers[a][b]-1);
            }
            printf("\n");
        }
        printf("\n");
        for (int a = 0; a < K; a++)
        {
            printf("I Center Diff %d - %d\t: ", a, a+1);
            for (int b = 0; b < i_centers_diff[a].size(); b++)
            {
                printf("%d ", i_centers_diff[a][b]-1);
            }
            printf("\n");
        }
        printf("\n");
    }
 
    vector<vector<int>> i_centers_min_dist_vert;
    bool bunches[N+1][N+1];
    memset(&bunches, 0, sizeof(bunches));
    create_i_centers_min_dist_vert__bunches(i_centers, i_centers_diff, i_centers_min_dist_vert, bunches, dist_mat);

    auto stop = std::chrono::high_resolution_clock::now();
 
    if (PRINT_STEPS)
    {
        for (int a = 1; a <= N; a++)
        {
            printf("Vertex %d\t: ", a-1);
            for (int b = 0; b < K; b++)
            {
                printf("%d ", i_centers_min_dist_vert[a][b]-1);
            }
            printf("\n");
        }
        printf("\n");
        for (int a = 1; a <= N; a++)
        {
            printf("Vertex %d (Bunches)\t: ", a-1);
            for (int b = 1; b <= N; b++)
            {
                if (bunches[a][b])
                {
                    printf("%d ", b-1);
                }
            }
            printf("\n");
        }
        printf("\n");

        for (int a = 1; a <= N; a++)
        {
            printf("0 to %d: CALC = %lld AND ACT = %lld\n", a, get_approximate_distance(1, a, bunches, i_centers_min_dist_vert, dist_mat), dist_mat[1][a]);
        }
    }

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(stop - start);
    printf("\nTime Taken For Thorup-Zwick Algorithm: %ld microseconds\n", duration);
}

Overwriting Distance_Oracle_Serial.cpp


In [74]:
%%shell

g++ Distance_Oracle_Serial.cpp -o Distance_Oracle_Serial
./Distance_Oracle_Serial

Distance_Oracle_Serial.cpp: In function ‘int main()’:
Distance_Oracle_Serial.cpp:398:83: warning: format ‘%ld’ expects argument of type ‘long int’, but argument 2 has type ‘std::chrono::duration<long int, std::ratio<1, 1000000> >’ [-Wformat=]
     printf("\nTime Taken For Thorup-Zwick Algorithm: %ld microseconds\n", duration);
                                                                                   ^
Randomly Generated Graph:
V0: (V1, 40) (V2, 18) (V4, 6) (V5, 23) (V7, 42) (V8, 34) 
V1: (V0, 40) (V2, 9) (V5, 31) (V6, 29) (V8, 11) 
V2: (V0, 18) (V1, 9) (V5, 13) (V7, 20) (V8, 38) 
V3: (V5, 35) (V7, 27) (V8, 45) 
V4: (V0, 6) (V5, 3) (V6, 37) 
V5: (V0, 23) (V1, 31) (V2, 13) (V3, 35) (V4, 3) (V8, 47) 
V6: (V1, 29) (V4, 37) (V7, 34) 
V7: (V0, 42) (V2, 20) (V3, 27) (V6, 34) (V9, 36) 
V8: (V0, 34) (V1, 11) (V2, 38) (V3, 45) (V5, 47) (V9, 33) 
V9: (V7, 36) (V8, 33) 

Distance Matrix:
	V0	V1	V2	V3	V4	V5	V6	V7	V8	V9
V0	0	27	18	44	6	9	43	38	34	67	
V1	27	0	9	56	25	22	29	29	11	44	
V2	18	9	